In [1]:
# This is a practice Artificial Neural Network
# The problem being solved is based off of a model bank with fake data
# The bank has customers that have left for whatever reason
# The goal is to find why these customers have left using information such as account balance and gender
# The last column of the data states whether or not the customer has left the bank
import math
import time

import numpy as np #Math operations library
import matplotlib.pyplot as plt #Visualization library
import pandas as pd #Matrix handler

import keras # Brings in tensorflow with it
from keras.models import Sequential # Used for initialization of ANN
from keras.layers import Dense, Dropout# adds layers to ANN
from keras.wrappers.scikit_learn import KerasClassifier # ability to turn network into a function definition
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

from sklearn.preprocessing import StandardScaler #Methods to change categorical strings to numbers and scaling ability

parameters = {'batch_size': 16, # 50 epochs, 264 nodes, 3 hidden layers
              'epochs': 100,
              'learning_rate': 0.00001,
              'nodes': 264,
              'hidden_layers': 3,
              'dropout': 0.5
             } # Creates list of parameters to test to find most successful one
parameters['optimizer'] = Adam(parameters['learning_rate'])

class data_handler:
    def __init__(self, pkl_location, sc):
        data = pd.read_pickle(pkl_location)
        data = data.sample(frac=1).reset_index(drop=True)
        
        x = data[list(range(0, 264))]# max 264 x columns
        x = sc.fit_transform(x)
        self.x = pd.DataFrame(x)
        
        self.y = data[['Rest', 'Emote', 'Solve']]
        del data
        
        self.tenth = math.floor(self.x.shape[0] / 10)
        
    def get_cross_val(self, iteration):
        if iteration > 10:
            raise ValueError('Crossval Iteration Exceeds 10')
        
        iteration = iteration - 1
        train_array = list(range(0, iteration*self.tenth)) + list(range((iteration+1)*self.tenth, self.x.shape[0]))
        test_array = list(range(iteration*self.tenth, (iteration+1)*self.tenth))
        
        x_train = self.x.iloc[train_array].values
        y_train = self.y.iloc[train_array].values
        x_test = self.x.iloc[test_array].values
        y_test = self.y.iloc[test_array].values
        
        return x_train, y_train, x_test, y_test

def build_classifier(parameters):
    classifier = Sequential() # This is the ANN object
    classifier.add(Dense(input_dim=264, units=parameters['nodes'], kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=parameters['dropout']))
    
    for layer in range(0, parameters['hidden_layers']):
        classifier.add(Dense(units=parameters['nodes'], kernel_initializer='uniform', activation='relu')) #Creates first hidden layer
        classifier.add(Dropout(rate=parameters['dropout']))
        
    classifier.add(Dense(units=3, kernel_initializer='uniform', activation='softmax')) # Output layer. Only 1 ouput category, sigmoid activation to get probability of sureness
    # Note: Softmax applies to a dependent variable that has more than 2 categories
    # i.e. fMRI categorizations
    
    classifier.compile(loss='categorical_crossentropy',
              optimizer=parameters['optimizer'],
              metrics=['accuracy'])
    # Notes
    # adam is a kind of stochastic gradient descent
    # For multivariabel, use categorical cross entropy
    # Accuracy is predefined
    return classifier
# Creates a standard Keras type classifier composed of the defined network for
# k-means testing
data = data_handler('./atlas.pkl', StandardScaler())

Using TensorFlow backend.


In [ ]:

for i in range(0, 10):
    i = i + 1
    #train
    tensorboard = TensorBoard(log_dir="../logs/{}".format(time.time()))
    x_train, y_train, x_test, y_test = data.get_cross_val(i)
    classifier = build_classifier(parameters)
    classifier.fit(x_train, y_train, batch_size=parameters['batch_size'], epochs=parameters['epochs'],callbacks=[tensorboard], validation_split=0.5)
    
    #test
    y_pred = classifier.predict(x_test)
    y_pred = pd.DataFrame(y_pred)
    y_test = pd.DataFrame(y_test)
    print(y_test)
    y_pred['test_0'] = y_test.iloc[:, [0]]
    y_pred['test_1'] = y_test.iloc[:, [1]]
    y_pred['test_2'] = y_test.iloc[:, [2]]
    
    y_pred.to_pickle('./Cross_Validation_Results/cv_results_' + str(i - 1) + '.pkl')
    


Train on 207847 samples, validate on 207847 samples
Epoch 1/100
207847/207847 [==============================] - 48s 232us/step - loss: 0.8546 - acc: 0.5981 - val_loss: 0.8061 - val_acc: 0.5990
Epoch 2/100
207847/207847 [==============================] - 49s 237us/step - loss: 0.8053 - acc: 0.6017 - val_loss: 0.8051 - val_acc: 0.5993
Epoch 3/100
207847/207847 [==============================] - 49s 234us/step - loss: 0.8043 - acc: 0.6018 - val_loss: 0.8044 - val_acc: 0.5993
Epoch 4/100
207847/207847 [==============================] - 50s 240us/step - loss: 0.8032 - acc: 0.6018 - val_loss: 0.8028 - val_acc: 0.5992
Epoch 5/100
207847/207847 [==============================] - 49s 237us/step - loss: 0.8010 - acc: 0.6028 - val_loss: 0.7986 - val_acc: 0.6022
Epoch 6/100
207847/207847 [==============================] - 50s 242us/step - loss: 0.7948 - acc: 0.6129 - val_loss: 0.7885 - val_acc: 0.6201
Epoch 7/100
207847/207847 [==============================] - 51s 244us/step - loss: 0.7863 - acc

In [ ]:
print(classifier.summary())